In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from IPython.core.pylabtools import figsize
import os

import random

from math import pi
import time

import torch
import torchvision
from torchvision import datasets
import torchvision.transforms as T
import albumentations as A


from utils.augmentation_utils import *
from utils.FloatToChange import FloatToChange

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# crop_transperent_fields(logo_dir) 

In [4]:
WIDTH = 100
HEIGHT = 100
FPS = 25

In [17]:
# random text generator

h, w = 64, 128
output_path = r"data_synthetic\9_spray_only"

text_source = r"data_synthetic\sample_text.txt"
font_source = r"data\fonts"

for _ in range(10000):

    img = draw_text(h, w, font_source, text_source, convert_RGB=False)
    image_writer(output_path, img, "0")

In [ ]:

bg_transforms_vid = A.Compose([
                                A.Blur(blur_limit=(3,FloatToChange(5) + 3),p=FloatToChange(0.77)),
                                A.ColorJitter(brightness=[-FloatToChange(0.3) + 1, FloatToChange(0.3) + 1], contrast=[- FloatToChange(0.3)+1,FloatToChange(0.3)+1], saturation=[-FloatToChange(0.1)+1,FloatToChange(0.1)+1], hue=[-FloatToChange(0.07),FloatToChange(0.07)], always_apply=False, p=FloatToChange(1)),
                                A.ISONoise(color_shift=(0, FloatToChange(0.3)), intensity=(0, FloatToChange(0.3)), p=FloatToChange(1)),
                                A.Sharpen (alpha=(0, FloatToChange(1)), lightness=(-FloatToChange(1)+1,FloatToChange(1)+1), p=FloatToChange(1)),
                            ])

bg_transforms_pic = A.Compose(np.hstack((list(bg_transforms_vid),[ #way to avoid nested Compose objects inside another Compose
                                A.Rotate(limit=2,crop_border=True,p=1),
                                A.RandomResizedCrop(height=HEIGHT, width=WIDTH,scale=(-FloatToChange(0.05)+1, 1),ratio=[1,1]),
                            ])))

final_transforms = A.Compose([
                                A.Blur(blur_limit=(3,FloatToChange(11) + 3),p=FloatToChange(0.77)),
                                A.Downscale(scale_min= -FloatToChange(0.77)+.999,scale_max=.999, interpolation=cv2.INTER_NEAREST, always_apply=True, p=FloatToChange(1)),
                                A.ColorJitter(brightness=[-FloatToChange(0.4) + 1, FloatToChange(0.4) + 1], contrast=[- FloatToChange(0.4)+1,FloatToChange(0.4)+1], saturation=[-FloatToChange(0.9)+1,FloatToChange(0.9)+1], hue=[-FloatToChange(0.5),FloatToChange(0.5)], always_apply=False, p=FloatToChange(1)),
                                A.ISONoise(color_shift=(0, FloatToChange(0.3)), intensity=(0, FloatToChange(0.3)), p=FloatToChange(1)),
                                A.ImageCompression(quality_lower=-FloatToChange(100)+100, quality_upper=100, always_apply=False, p=FloatToChange(1)),
                                A.PixelDropout (dropout_prob=FloatToChange(0.33), per_channel=True, drop_value=0, mask_drop_value=None, always_apply=False, p=FloatToChange(0.33)),
                                A.RingingOvershoot (blur_limit=(7, (FloatToChange(3) + 7).as_int()), cutoff=(-FloatToChange(0.125)*pi + 3/8*pi, FloatToChange(0.125)*pi + 3/8*pi), p=FloatToChange(1)),
                                A.Sharpen (alpha=(0, FloatToChange(1)), lightness=(-FloatToChange(1)+1,FloatToChange(1)+1), p=FloatToChange(1)),
                                A.UnsharpMask (blur_limit=(3, (FloatToChange(11) + 3).as_int()), sigma_limit=(0, FloatToChange(11)), alpha=(0, FloatToChange(1)), always_apply=False, p=FloatToChange(0.8)),
                            ])

bg_transforms_vid = AugTransform(bg_transforms_vid, FloatToChange)
bg_transforms_pic = AugTransform(bg_transforms_pic, FloatToChange)
final_transforms = AugTransform(final_transforms, FloatToChange)

In [ ]:
# for single logo image augmentator

images_dir = r'data\real_data\auto_labeled_data\0'
logo_dir   = r'data\194_logo_icons\png\3m'
synthetic_dir = r'data\194_logo_icons\png'


img_generator = LogoImageGenerator(images_dir, logo_dir, synthetic_dir, WIDTH, HEIGHT, 
                                logo_name="3m", bg_transform=bg_transforms_pic, 
                                final_transforms=final_transforms, dr_graphical=0.7, 
                                dr_geometrical=1, logo_base_scale=0.8, print_state=False)
img_generator(200, with_logo=True)

In [ ]:
# for bunch of logos logo image augmentator

# images_dir = r'C:\Users\dan\Desktop\style_transfer\datasets\rampage_dataset\0'
images_dir = r'C:\Users\dan\Desktop\vcr_logo_detector\data_synthetic\synthetic_text\for_simclr'
logo_dir   = r'data\194_logo_icons\png'
output_dir = r'data\194_logo_icons'

filenames = next(os.walk(logo_dir), (None, None, []))[2]  # files only

for file_name in filenames:
    new_folder = ".".join(file_name.split(".")[:-1])
    logo_path = os.path.join(logo_dir, file_name)
    img_generator = LogoImageGenerator(images_dir, logo_path, output_dir, WIDTH, HEIGHT, 
                                    logo_name=new_folder, bg_transform=bg_transforms_pic, 
                                    final_transforms=final_transforms, dr_graphical=0.5, 
                                    dr_geometrical=0.8, logo_base_scale=0.8, print_state=False,
                                    create_subfolder=False)
    img_generator(1000, with_logo=True)

In [ ]:
synthetic_video_dir = r'synthetic_video_dataset'

generator = LogoSequenceGenerator(images_dir, logo_dir, synthetic_video_dir, bg_transforms=bg_transforms_vid, final_transforms=final_transforms)
generator(3)

In [ ]:
# sample from big text images 
img_folder = r"data_synthetic\synthetic_text\xx"
random_crop = A.RandomResizedCrop (100, 100, scale=(0.1, 0.4), p=1.0)
for _ in range(20000):
    img_path = sample_random_file(img_folder)
    image = cv2.imread(img_path)
    image = random_crop(image=image)['image']
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_writer(output_path, image, "xx_crop")


In [ ]:
# replace the very CNN letters
img_folder = r"C:\Users\Admin\Desktop\vcr_research\vcr_logo_detector\data\real_data\auto_labeled_data\1"
output_path = r"C:\Users\Admin\Desktop\vcr_research\vcr_logo_detector\data"

for _ in range(9500):
    img_path = sample_random_file(img_folder)
    image = cv2.imread(img_path)
    if image.shape[0] < 100: continue
    image[42:76, 22:78, :] = image[42, 22, :]

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_writer(output_path, image, "patches_0_attack_minus_cnn")